In [1]:
import io

import requests
import docx

In [2]:
def clean_line(line):
    line = line.strip()
    line = line.strip('\uFEFF')
    return line

def read_faq(file_id):
    url = f'https://docs.google.com/document/d/{file_id}/export?format=docx'
    
    response = requests.get(url)
    response.raise_for_status()
    
    with io.BytesIO(response.content) as f_in:
        doc = docx.Document(f_in)
    
    questions = [] 

    question_heading_style = 'heading 2'
    section_heading_style = 'heading 1'

    heading_id = ''
    section_title = ''
    question_title = ''
    answer_text_so_far = ''

    for p in doc.paragraphs:
        style = p.style.name.lower()
        p_text = clean_line(p.text)

        if len(p_text) == 0:
            continue

        if style == section_heading_style:
            answer_text_so_far = answer_text_so_far.strip()
            if answer_text_so_far != '' and section_title != '' and question_title != '':
                questions.append({
                    'text': answer_text_so_far,
                    'section': section_title,
                    'question': question_title
                })
                answer_text_so_far = ''

            question_title = p_text
            continue

        answer_text_so_far += '\n' + p_text

    answer_text_so_far = answer_text_so_far.strip()
    if answer_text_so_far != '' and section_title != '' and question_title != '':
        questions.append({
            'text': answer_text_so_far,
            'section': section_title,
            'question': question_title
        })

    return questions
    

In [3]:
faq_documents = {
    'data-engineering-zoomcamp': '19bnYs80DwuUimHM65UV3sylsCn2j1vziPOwzBwQrebw',
    'machine-learning-zoomcamp': '1LpPanc33QJJ6BSsyxVg-pWNMplal84TdZtq10naIhD8',
    'mlops-zoomcamp': '12TlBfhIiKtyBv8RnsoJR6F72bkPDGEvPOItJIxaEzE0'
}

In [4]:
documents = []

for course, file_id in faq_documents.items():
    print(course)
    course_documents = read_faq(file_id)
    documents.append({
        'course': course,
        'documents': course_documents
    })

data-engineering-zoomcamp


machine-learning-zoomcamp
mlops-zoomcamp


In [5]:
import json

In [6]:
with open('documents.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [7]:
!head documents.json

[
  {
    "course": "data-engineering-zoomcamp",
    "documents": []
  },
  {
    "course": "machine-learning-zoomcamp",
    "documents": []
  },
  {
